#### All imports here:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Reading data set

In [ ]:
train_data = pd.read_csv("./datasets/history.csv")
train_data.head()

In [ ]:
info_df = pd.read_csv("account_data_info.csv")
pd.set_option('display.max_colwidth', None)
info_df

#### Dropping Columns and Research

In [ ]:
# Aditya Parekh:
# I am removing these columns because they have 600k+ NULL values.
rem_cols_na = ['country_code', 'cashflows_custody_fee', 'arp_pension_origin', 'sss_location']
# I am removing these columns because they have 0 NULL values, but only one unique value.
rem_cols_one_uniq = ['net_of_fees', 'fee_paid_separately', 'custody_fee_withdrawal', 'is_fee_exempt']
# The following columns are NULL when 'class_id' = NULL. DON'T REMOVE THESE YET.
null_3198_cols = ['debit_code', 'last_trade_date', 'contract_type', 'inception_date',
             'branch', 'credit_limit_type', 'retail_plan', 'language_code', 'dividend_confirm_code']

In [ ]:
# Aditya Mani:
# 1. esir_number: Contains 673327 missing values and cannot be imputed (has very little unique values)
# 2. shareholder_language: Contains 623554 missing values. Says whether the shareholder speaks English or French and there seems to be an equal proportion.
# 3. has_no_min_commission: No Null values and only 1 unique value.
# 4. rep_commission_rate: 1 unique value and few missing values that cannot be imputed.
# 5. spousal_age_flag: No Null values and only 1 unique value.
# 6. is_parameters_account: No Null values and only 1 unique value.
# 7. rrsp_limit_reached: No Null values and only 1 unique value.
# 8. is_portfolio_account: No Null values and only 1 unique value.
# 9. sss_type: Lot of missing values
# 10. sss_agent: Lot of missing values
# 11. target_grantor_grantee_flag: Missing values
# 12. terminal_code: Missing values

to_drop = ['esir_number', 'shareholder_language', 'has_no_min_commission', 'rep_commission_rate', 
           'spousal_age_flag', 'is_parameters_account', 'rrsp_limit_reached', 'is_portfolio_account',
             'sss_type', 'sss_agent', 'target_grantor_grantee_flag', 'terminal_code']

missing_3198 = ['options_trading_type',
 'is_midwest_clearing_account',
 'rep_commission_override',
 'interest_dividend_conversion_type',
 'guarantee_gtor_type',
 'deceased_fair_market_value',
 'iso_funds_code']

In [ ]:
# Jithin Krishna:

# Has too many null values
# last_update_date is the maintaineance date and was deemed not relevant to the churn
drop_cols = ["special_tag", "conjunction", "title", "function_code" ,"tms_settlement_location", "loan_limit_override", "last_update_date"]

# These values have 1 unique value and is not useful and hence shall be dropped
drop_uniq_cols = ["portfolio_cost_method", "portfolio_name_address_option", "portfolio_summary_option","interactive_portfolio_code"]

drop_3198_cols = ["portfolio_report_option", "mailing_consent"]

In [ ]:
# Srishti Prayag:
# remove columns with more than 300,000 missing values
too_many_nans = ['special_fee_code', 'resp_specimen_plan', 'plan_end_date', 'retail_last_maintenance_user']

# list of columns with exactly 316985 missing values; may be connected so don't drop yet
nan_316985_columns = ['is_pledged', 'non_calendar_year_end', 'plan_effective_date', 'is_resp', 'number_of_beneficiaries', 'rrif_original_date', 'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 'is_plan_grandfathered', 'retail_last_maintenance_time']

# list of columns with only 1 unique value
only_one_unique = ['is_plan_grandfathered']

In [ ]:
rem_cols = []
rem_cols.append(rem_cols_na)
rem_cols.append(rem_cols_one_uniq)
rem_cols.append(to_drop)
rem_cols.append(drop_cols)
rem_cols.append(drop_uniq_cols)
rem_cols.append(too_many_nans)
train_data = train_data.drop(rem_cols, axis=1)